In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import col, regexp_extract,to_date, year, month, dayofmonth


# Setup iceberg config
conf = SparkConf().setAppName("MyApp") \
    # .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    # .set(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    # .set('spark.jars', iceberg_spark_jar) \
    # .set(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    # .set(f"spark.sql.catalog.{catalog_name}.type", "hadoop")\
    # .set("spark.sql.defaultCatalog", catalog_name) 

# Create spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print("done")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 09:15:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


done


In [2]:
# read parquet file
df = spark.read.parquet("hdfs://namenode:9000/iceberg/db/table/data/year=2023/month=1/day=1")

# show schema
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Date_Joined: date (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Selector_Investors: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [3]:

# show data frame
df.show()

+-----+---------------+---------+-----------+--------------------+--------+------------------+--------------------+----+-----+---+
|index|        Company|Valuation|Date_Joined|             Country|    City|          Industry|  Selector_Investors|year|month|day|
+-----+---------------+---------+-----------+--------------------+--------+------------------+--------------------+----+-----+---+
|  618|          Tabby|    $1.50| 2023-01-01|United Arab Emirates|   Dubai|Financial Services|Arbor Ventures, S...|2023|    1|  1|
|  823|     MiniMax AI|     $1.2| 2023-01-01|               China|Shanghai|   Enterprise Tech|HongShan, IDG Cap...|2023|    1|  1|
|  830|Our Next Energy|    $1.20| 2023-01-01|       United States|    Novi|       Industrials|BMW i Ventures, B...|2023|    1|  1|
|  896|          Halan|       $1| 2023-01-01|               Egypt|   Cairo|Financial Services|Middle East Ventu...|2023|    1|  1|
|  618|          Tabby|    $1.50| 2023-01-01|United Arab Emirates|   Dubai|Financia

In [ ]:
df_partition = spark.read.parquet(
    "hdfs://namenode:9000/user/hive/warehouse/web_logs_partitioned/year=2019/month=1/day=22"
)
df_partition.show()


In [ ]:
streaming_df = spark.readStream \
    .format("parquet") \
    .option("path", "hdfs://namenode:9000/user/hive/warehouse/web_logs_partitioned") \
    .load()
print("done")

In [ ]:
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

In [ ]:
query.awaitTermination()